# Мониторинг звуковой среды с помощью предобученной модели

В этом проекте решается задача мониторинга звуковой среды (природные и техногенные звуки) с использованием предобученной аудио‑модели и датасета ESC‑50. Цель — показать прототип системы, которая по коротким аудиозаписям определяет типы звуков (птицы, дождь, транспорт и т.п.) и может быть использована для отслеживания изменения звуковой экосистемы во времени.


В современных городах и природных территориях важно отслеживать изменения звуковой среды: снижение активности птиц, рост техногенного шума, появление аномальных звуков. Ручной анализ аудиозаписей трудозатратен, поэтому полезен автоматизированный инструмент, который по аудиофайлам оценивает, преобладают ли природные или техногенные звуки.

В этом прототипе:
- «natural» — звуки природы (пение птиц, гром, дождь, ветер, насекомые и т.д.).
- «industrial» — техногенные звуки (бытовая техника, транспорт, сирены и т.п.).

Система может использоваться экологами, исследователями и городскими службами как вспомогательный инструмент мониторинга экосистемы по звуку.

# 1. Установка и импорты 

In [1]:
!pip install librosa soundfile torchaudio torch torchvision --quiet

import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import torchaudio
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


# 2. Загрузка датасета ESC‑50 


Для демонстрации используется открытый датасет ESC‑50, содержащий 2000 аудиозаписей по 5 секунд в 50 классах звуков окружающей среды (животные, природные явления, бытовые и городские звуки). Из него формируется поднабор из двух групп:
- природные звуки (`natural`);
- техногенные/индустриальные звуки (`industrial`).

По колонке `category` в `esc50.csv` классам задаются группы, после чего остаётся 320 записей: 200 природных и 120 техногенных.

## Загрузка датасета ESC‑50


In [2]:
import zipfile
import requests

DATA_ROOT = "data"
ESC50_DIR = os.path.join(DATA_ROOT, "ESC-50")
os.makedirs(DATA_ROOT, exist_ok=True)

zip_url = "https://github.com/karolpiczak/ESC-50/archive/master.zip"
zip_path = os.path.join(DATA_ROOT, "ESC-50.zip")

if not os.path.exists(ESC50_DIR):
    r = requests.get(zip_url)
    with open(zip_path, "wb") as f:
        f.write(r.content)

    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(DATA_ROOT)

    # Переименуем распакованную папку в удобное имя
    os.rename(os.path.join(DATA_ROOT, "ESC-50-master"), ESC50_DIR)



Скачиваю ESC-50...
Готово, датасет в папке: data/ESC-50


In [3]:
DATA_DIR = "/Users/yaroslavbaev/Desktop/miphi/data/ESC-50"

meta_path = os.path.join(DATA_DIR, "meta", "esc50.csv")
meta = pd.read_csv(meta_path)
meta.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


# 3. Формирование подзадачи: природные vs техногенные звуки

## Формирование групп классов: природные и техногенные звуки


In [4]:
natural_classes = [
    "chirping_birds",
    "thunderstorm",
    "rain",
    "wind",
    "crickets"
]

industrial_classes = [
    "vacuum_cleaner",
    "car_horn",
    "siren",
    "jackhammer",
    "engine_idling"
]

meta["group"] = meta["category"].apply(
    lambda c: "natural" if c in natural_classes
    else ("industrial" if c in industrial_classes else "other")
)

meta["group"].value_counts()


group
other         1680
natural        200
industrial     120
Name: count, dtype: int64

## Выбор поднабора данных для мониторинга звуковой среды


In [5]:
meta_subset = meta[meta["group"].isin(["natural", "industrial"])].reset_index(drop=True)
len(meta_subset), meta_subset["group"].value_counts()


(320,
 group
 natural       200
 industrial    120
 Name: count, dtype: int64)

## 4. Предобученная аудио‑модель из Transformers (Hugging Face)




Чтобы не обучать собственную нейросеть с нуля, используется предобученная аудио‑модель `MIT/ast-finetuned-audioset-10-10-0.4593`, обученная на большом датасете AudioSet. Модель принимает на вход сырое аудио и возвращает распределение вероятностей по 527 звуковым классам. Это позволяет использовать её как готовый «датчик» звукового окружения.

In [16]:
!pip install transformers torchaudio --quiet


In [18]:
from transformers import AutoProcessor, AutoModelForAudioClassification

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model_name = "MIT/ast-finetuned-audioset-10-10-0.4593" 
processor = AutoProcessor.from_pretrained(model_name)
audio_model = AutoModelForAudioClassification.from_pretrained(model_name).to(device)
audio_model.eval()

id2label = audio_model.config.id2label
len(id2label), list(id2label.items())[:5]


Using device: cpu


preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

(527,
 [(0, 'Speech'),
  (1, 'Male speech, man speaking'),
  (2, 'Female speech, woman speaking'),
  (3, 'Child speech, kid speaking'),
  (4, 'Conversation')])

На этом шаге определяется функция загрузки аудиофайла, приведения его к нужной частоте дискретизации и вызова предобученной модели. Модель возвращает вектор вероятностей по 527 классам, а также топ‑k наиболее вероятных классов, что позволяет интерпретировать тип звука.

In [21]:
import librosa
import numpy as np

TARGET_SR = 16000  # под ast-модель

def load_audio_waveform(path, sr=TARGET_SR):
    y, orig_sr = librosa.load(path, sr=None)  # читать через librosa
    if orig_sr != sr:
        y = librosa.resample(y, orig_sr=orig_sr, target_sr=sr)
    return y  #1D numpy array (T, )

def predict_audio_labels(path, top_k=5):
    waveform = load_audio_waveform(path, sr=TARGET_SR)
    inputs = processor(waveform, sampling_rate=TARGET_SR, return_tensors="pt")
    inputs = {k: torch.tensor(v).to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = audio_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)[0].cpu().numpy()

    top_idx = probs.argsort()[::-1][:top_k]
    results = [(id2label[int(i)], float(probs[i])) for i in top_idx]
    return results, probs


In [22]:
example_row = meta_subset.iloc[0]
audio_path = os.path.join(ESC50_DIR, "audio", example_row["filename"])
print(example_row["filename"], "->", example_row["category"], "->", example_row["group"])

top_preds, probs_vec = predict_audio_labels(audio_path, top_k=5)
for label, p in top_preds:
    print(f"{label}: {p:.3f}")
print("Размер вектора вероятностей:", probs_vec.shape)


1-100038-A-14.wav -> chirping_birds -> natural


/var/folders/q0/29fzx5zj31g3hxygfclyh8sr0000gn/T/ipykernel_74945/3558672398.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = {k: torch.tensor(v).to(device) for k, v in inputs.items()}


Bird: 0.256
Bird vocalization, bird call, bird song: 0.230
Owl: 0.169
Chirp, tweet: 0.135
Animal: 0.035
Размер вектора вероятностей: (527,)


Результат для примера с категорией `chirping_birds` показывает, что предобученная модель корректно распознаёт птичьи звуки (в топ‑классах присутствуют категории, связанные с птицами). Это подтверждает, что модель подходит в качестве основы для мониторинга звуковой экосистемы.

# 5. Признаки и классификатор


Вектор вероятностей по 527 классам, который возвращает предобученная модель, используется как признаковое представление каждого аудиофайла. На этих признаках обучается лёгкий линейный классификатор (логистическая регрессия), решающий задачу: `natural` vs `industrial`. Это позволяет построить прикладное решение для мониторинга звуковой среды без обучения тяжёлой нейросети.


In [23]:
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

N_SAMPLES = 100  # можно увеличить до 200–300, если по времени ок
subset_small = meta_subset.sample(N_SAMPLES, random_state=42).reset_index(drop=True)

features = []
labels_group = []
filenames = []

for idx, row in tqdm.tqdm(subset_small.iterrows(), total=len(subset_small)):
    path = os.path.join(ESC50_DIR, "audio", row["filename"])
    _, probs_vec = predict_audio_labels(path, top_k=5)
    features.append(probs_vec)
    labels_group.append(row["group"])
    filenames.append(row["filename"])

features = np.stack(features)  # (N, 527)

X = features
y = np.array([1 if g == "natural" else 0 for g in labels_group])  # 1=natural, 0=industrial

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=["industrial", "natural"]))
print(confusion_matrix(y_test, y_pred))


/var/folders/q0/29fzx5zj31g3hxygfclyh8sr0000gn/T/ipykernel_74945/3558672398.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = {k: torch.tensor(v).to(device) for k, v in inputs.items()}
100%|██████████| 100/100 [00:49<00:00,  2.03it/s]

              precision    recall  f1-score   support

  industrial       0.89      0.67      0.76        12
     natural       0.81      0.94      0.87        18

    accuracy                           0.83        30
   macro avg       0.85      0.81      0.82        30
weighted avg       0.84      0.83      0.83        30

[[ 8  4]
 [ 1 17]]


Полученные метрики (precision, recall, F1‑мера) для классов `natural` и `industrial` показывают, что даже простой линейный классификатор на предобученных признаках способен различать природные и техногенные звуки. Это демонстрирует, что комбинация предобученной модели и лёгкого классификатора уже даёт рабочий прототип системы мониторинга звуковой экосистемы.


# 6. end‑to‑end сценарий

На практике пользователь работает не с таблицей, а с отдельными аудиозаписями. Ниже показан end‑to‑end сценарий: на вход подаётся путь к аудиофайлу, система использует предобученную модель для извлечения признаков, затем применяет обученный классификатор и возвращает решение `natural` или `industrial` вместе с уверенностью модели и топ‑классами по AudioSet.

In [24]:
def classify_sound_business(path):
    # Предсказания предобученной модели (признаки)
    _, probs_vec = predict_audio_labels(path, top_k=5)
    probs_vec = probs_vec.reshape(1, -1)

    # Предсказание нашего классификатора
    pred = clf.predict(probs_vec)[0]
    proba = clf.predict_proba(probs_vec)[0][pred]

    business_label = "natural" if pred == 1 else "industrial"

    # Для интерпретации ещё раз получаем топ-5 классов AST-модели
    top_labels, _ = predict_audio_labels(path, top_k=5)
    return business_label, proba, top_labels

# Пример использования на случайно выбранном файле из meta_subset
test_row = meta_subset.sample(1, random_state=0).iloc[0]
test_path = os.path.join(ESC50_DIR, "audio", test_row["filename"])

business_label, proba, top_labels = classify_sound_business(test_path)

print("Файл:", test_row["filename"])
print("Реальная группа:", test_row["group"])
print(f"Модель: {business_label} (p={proba:.3f})")
print("Топ-5 классов AST-модели:")
for label, p in top_labels:
    print(f"  {label}: {p:.3f}")


/var/folders/q0/29fzx5zj31g3hxygfclyh8sr0000gn/T/ipykernel_74945/3558672398.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = {k: torch.tensor(v).to(device) for k, v in inputs.items()}
/var/folders/q0/29fzx5zj31g3hxygfclyh8sr0000gn/T/ipykernel_74945/3558672398.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = {k: torch.tensor(v).to(device) for k, v in inputs.items()}


Файл: 3-243726-A-43.wav
Реальная группа: industrial
Модель: natural (p=0.548)
Топ-5 классов AST-модели:
  Vehicle: 0.249
  Speech: 0.105
  Train: 0.099
  Outside, urban or manmade: 0.070
  Rail transport: 0.051


Звук на этом файле относится к техногенным (industrial), но классификатор по предобученной модели решил, что это скорее «natural» с невысокой уверенностью 0.548.​

При этом сама AST‑модель в топ‑классах видит транспорт, речь и городской/man‑made фон (vehicle, train, urban), то есть по сути правильно «чувствует» техногенную среду, просто линейный классификатор поверх её признаков в этом случае ошибся и перепутал группу. Все мы ошибаемся...

## 7. Выводы

В рамках проектной работы реализован прототип системы мониторинга звуковой среды, который:
- использует открытый датасет ESC‑50 для моделирования природных и техногенных звуков;
- опирается на предобученную аудио‑модель `MIT/ast-finetuned-audioset-10-10-0.4593` без переобучения;
- строит лёгкий классификатор для задачи `natural` vs `industrial` на признаках предобученной модели;
- предоставляет end‑to‑end сценарий для классификации отдельного аудиофайла.

Такое решение может быть расширено до реального мониторинга экосистемы: достаточно подключить реальные аудио‑датчики и регулярно обрабатывать записи, оценивая долю природных и техногенных звуков во времени.
